# Criterios de projecto Final
Entrega: 8 de Mayo
Requerimientos:

* Descargar o obtener un dataset de su interes
* Tiene que ser una aplicación desarrollada en streamlit
* Tiene que poder generar un reporte en PDF y poder descargarlo desde la aplicación
* Tiene que poder generar un reporte en Excel y poder descargarlo desde la aplicación
* Tiene que tener gráficos interactivos (Dashboard) (Filtros dependiendo de la data)

# Objetivo:
Encontrar el nmero de pacientes en ciertos rangos de edad que es primera vez y prevención.
* Menor a 1 año.
* De 1 a 4 años.
* De 5 a 9 años.
* De 10 a 14 años.
* De 15 a 19 años.
* De 20 a 64 años.
* De 65 años y más.

Nota: Crear gráficos al inicio del programa

In [1]:
import pandas as pd

df = pd.read_excel('Datasets/000_04D02_PRAS_ENERO_MARZO_2024.xlsx')

In [2]:
df.head()

,ENT_ID,ENT_RUC,ENT_NOM,ENT_SIM_TIP_EST,ENT_DES_TIP_EST,ENT_COD_PROV,ENT_DES_PROV,ENT_COD_CANT,ENT_DES_CANT,ENT_COD_PARR,...,PCTE_ULT_IMC_CATEGORIA,SEMANAS,NIVEL_DESCRIPCION,COD_TIPO_ENTIDAD,TIPO_ENTIDAD,IGU_DESCRIPCION,ATEMED_CIE10 | ATEMED_DES_CIE10,ATENCION_DIA,ATENCION_MES,ATENCION_AÑO
0,236,460003580001,BOLIVAR,CS-A,CENTRO DE SALUD TIPO A,4,CARCHI,402,BOLIVAR,40250,...,Sobrepeso,NaN,NIVEL 1,209,MSP,MSP,R51X | CEFALEA,2,1,2024
1,236,460003580001,BOLIVAR,CS-A,CENTRO DE SALUD TIPO A,4,CARCHI,402,BOLIVAR,40250,...,Sobrepeso,NaN,NIVEL 1,209,MSP,MSP,R252 | CALAMBRES Y ESPASMOS,3,1,2024
2,236,460003580001,BOLIVAR,CS-A,CENTRO DE SALUD TIPO A,4,CARCHI,402,BOLIVAR,40250,...,NaN,NaN,NIVEL 1,209,MSP,MSP,Z001 | CONTROL DE SALUD DE RUTINA DEL NIÑO,3,1,2024
3,236,460003580001,BOLIVAR,CS-A,CENTRO DE SALUD TIPO A,4,CARCHI,402,BOLIVAR,40250,...,NaN,NaN,NIVEL 1,209,MSP,MSP,Z012 | EXAMEN ODONTOLOGICO,4,1,2024
4,236,460003580001,BOLIVAR,CS-A,CENTRO DE SALUD TIPO A,4,CARCHI,402,BOLIVAR,40250,...,NaN,NaN,NIVEL 1,209,MSP,MSP,G804 | PARALISIS CEREBRAL ATAXICA,5,1,2024


In [11]:
df.index

RangeIndex(start=0, stop=124520, step=1)

In [182]:
keep_cols = ['PCTE_ANIOS','PCTE_MESES','PCTE_DIAS','ATEMED_TIP_DIAG','ATEMED_CRON_DIAG','ATENCION_MES','ATENCION_AÑO']
df_clean= df[df.columns.intersection(keep_cols)]

In [183]:
df_clean = df_clean[(df_clean['ATEMED_TIP_DIAG'] == 'Prevención') & (df_clean['ATEMED_CRON_DIAG'] == 'Primera')]

## 

In [184]:
df_clean['ATENCION_MES'].unique()

array([1, 2, 3], dtype=int64)

In [7]:
def edad_en_dias(anios,meses,dias):
    dias1 = dias
    if dias == -1:
        dias1 = 29
    return dias1+30*meses+12*30*anios

def rango_edad(anios,meses,dias):

    edad = edad_en_dias(anios,meses,dias)

    if edad < edad_en_dias(1,0,0):
        return 'Menor_1'
    if edad < edad_en_dias(5,0,0):
        return '1_a_4'
    if edad < edad_en_dias(10,0,0):
        return '5_a_9'
    if edad < edad_en_dias(15,0,0):
        return '10_a_14'
    if edad < edad_en_dias(20,0,0):
        return '15_a_19'
    if edad < edad_en_dias(65,0,0):
        return '20_a_64'
    if edad >= edad_en_dias(65,0,0):
        return 'Mayor_65'

In [185]:
df_clean['RANGO'] = df_clean.apply(lambda x: rango_edad(x.PCTE_ANIOS, x.PCTE_MESES,x.PCTE_DIAS), axis=1)

In [186]:
mes = 3
df_clean[df_clean['ATENCION_MES'] == mes].groupby(by = 'RANGO').count()[['PCTE_ANIOS']].reset_index()

,RANGO,PCTE_ANIOS
0,10_a_14,717
1,15_a_19,993
2,1_a_4,571
3,20_a_64,5350
4,5_a_9,1117
5,Mayor_65,1336
6,Menor_1,120


# Histograma

In [74]:
import numpy as np
import matplotlib.pyplot as plt

In [187]:
def graf_hist(mes,df_clean,file_name):
    if mes == 0:
        df_mes = df_clean.groupby(by = 'RANGO').count()[['PCTE_ANIOS']].reset_index()
        sort_rango = {'Menor_1':0, '1_a_4':1, '5_a_9':2, '10_a_14':3, '15_a_19':4, '20_a_64':5, 'Mayor_65':6}

        df_mes = df_mes.sort_values(by='RANGO', key=lambda x: x.map(sort_rango)).reset_index().drop('index',axis = 1)
        # Crear el gráfico de barras usando Matplotlib
        fig, ax = plt.subplots()
        label_edad = ['< 1', '1 - 4', '5 - 9', '10 - 14', '15 - 19', '20 - 64', '> 65']
        ax.bar(label_edad, df_mes['PCTE_ANIOS'],color='blue')  # Crea barras
        ax.set_xlabel('Edades (Años)')
        ax.set_ylabel('Número de personas')

        for i,j in list(zip(range(0,len(df_mes['PCTE_ANIOS'].index)),list(df_mes['PCTE_ANIOS']))):
            ax.text(i,j+100,f'{j}', ha = 'center')
        fig.savefig(f'{file_name}.png')
        plt.close(fig)
        return

    df_mes = df_clean[df_clean['ATENCION_MES'] == mes].groupby(by = 'RANGO').count()[['PCTE_ANIOS']].reset_index()
    sort_rango = {'Menor_1':0, '1_a_4':1, '5_a_9':2, '10_a_14':3, '15_a_19':4, '20_a_64':5, 'Mayor_65':6}

    df_mes = df_mes.sort_values(by='RANGO', key=lambda x: x.map(sort_rango)).reset_index().drop('index',axis = 1)

    # Crear el gráfico de barras usando Matplotlib
    fig, ax = plt.subplots()
    label_edad = ['< 1', '1 - 4', '5 - 9', '10 - 14', '15 - 19', '20 - 64', '> 65']
    ax.bar(label_edad, df_mes['PCTE_ANIOS'],color='blue')  # Crea barras
    ax.set_xlabel('Edades (Años)')
    ax.set_ylabel('Número de personas')

    for i,j in list(zip(range(0,len(df_mes['PCTE_ANIOS'].index)),list(df_mes['PCTE_ANIOS']))):
        ax.text(i,j+100,f'{j}', ha = 'center')
    fig.savefig(f'{file_name}.png')
    plt.close(fig)
    return

In [180]:
graf_hist(2,df_clean,'hist')

# Reporte en PDF

In [234]:
from fpdf import FPDF

class PDFWithBackground(FPDF):
    def __init__(self):
        super().__init__()
        self.background = None
 
    def set_background(self, image_path):
        self.background = image_path
 
    def add_page(self, orientation=''):
        super().add_page(orientation)
        if self.background:
            self.image(self.background, 0, 0, self.w, self.h)
 
    def footer(self):
        # Posición a 1.5 cm desde el fondo
        self.set_y(-15)
        # Configurar la fuente para el pie de página
        self.set_font('Arial', 'I', 8)
        # Número de página
        self.cell(0, 10, 'Página ' + str(self.page_no()), 0, 0, 'C')

In [250]:
list(df_clean['ATENCION_MES'].unique())

[1, 2, 3]

In [202]:
def mes_num(num):
    if num == 1:
        return 'enero'
    if num == 2:
        return 'febrero'
    if num == 3:
        return 'marzo'
    if num == 4:
        return 'abril'
    if num == 5:
        return 'mayo'
    if num == 6:
        return 'junio'
    if num == 7:
        return 'julio'
    if num == 8:
        return 'agosto'
    if num == 9:
        return 'septiembre'
    if num == 10:
        return 'octubre'
    if num == 11:
        return 'noviembre'
    if num == 12:
        return 'diciembre'

In [218]:
def edad_por_rango(mes,rango,df_clean):

    if mes == 0:
        df_mes = df_clean.groupby(by = 'RANGO').count()[['PCTE_ANIOS']].reset_index()

        return df_mes[df_mes['RANGO'] == rango]['PCTE_ANIOS'].iloc[0]

    df_mes = df_clean[df_clean['ATENCION_MES'] == mes].groupby(by = 'RANGO').count()[['PCTE_ANIOS']].reset_index()
    
    return df_mes[df_mes['RANGO'] == rango]['PCTE_ANIOS'].iloc[0]

In [254]:
pdf = PDFWithBackground()
pdf.set_background('Background/background.png')
pdf.add_page()

year = df_clean['ATENCION_AÑO'].iloc[0]

pdf.set_y(100)
pdf.set_font('Courier', style = 'B', size = 45)
pdf.multi_cell(190,15,'Coberturas de atención de las unidades operativas de primer nivel de la dirección distrital 04D02 Montufar Bolivar Salud',0,1,'L')

# Primera página del informe

pdf.add_page()

meses = df_clean['ATENCION_MES'].unique()

pdf.set_y(15)
pdf.set_font('Courier',style = 'B', size = 27) # Vienen por defecto, Arial, Times, Courier
pdf.multi_cell(170,10,f'Consultas de prevención en primer nivel de atención D04D02 Montúfar Bolívar Salud {mes_num(meses[0])}-{mes_num(meses[-1])} {year}',0,1,'R')

pdf.set_y(62)
pdf.set_font('Courier', size = 15) # Vienen por defecto, Arial, Times, Courier
pdf.multi_cell(180,6,f'El número de consultas en el periodo {mes_num(meses[0])}-{mes_num(meses[-1])} por rango de edades es:',0,1,'L')

etiquetas = ['Menores a 1 año','De 1 a 4 años','De 5 a 9 años','De 10 a 14 años','De 15 a 19 años','De 20 a 64 años','Mayores a 65 años']
llaves = ['Menor_1','1_a_4','5_a_9','10_a_14','15_a_19','20_a_64','Mayor_65']

lista = list(zip(etiquetas,llaves))

aux = 0

for etiqueta,llave in lista:
    pdf.set_y(80+aux)
    pdf.set_font('Courier', size = 15)
    pdf.multi_cell(180,6,f'{chr(149)} {etiqueta}: {edad_por_rango(0,llave)}',0,1,'L')
    aux += 6

graf_hist(0,df_clean,'hist')

pdf.image('hist.png',x=43,y=130,w=145)

for mes in meses:

    pdf.add_page()
    
    graf_hist(mes,df_clean,f'hist_{mes}')

    pdf.set_y(15)
    pdf.set_font('Courier',style = 'B', size = 27) # Vienen por defecto, Arial, Times, Courier
    pdf.multi_cell(170,10,f'Consultas de prevención en primer nivel de atención D04D02 Montúfar Bolívar Salud {mes_num(mes)} {year}',0,1,'R')

    pdf.set_y(62)
    pdf.set_font('Courier', size = 15) # Vienen por defecto, Arial, Times, Courier
    pdf.multi_cell(180,6,f'El número de consultas en {mes_num(mes)} por rango de edades es:',0,1,'L')

    aux = 0

    for etiqueta,llave in lista:
        pdf.set_y(80+aux)
        pdf.set_font('Courier', size = 15)
        pdf.multi_cell(180,6,f'{chr(149)} {etiqueta}: {edad_por_rango(mes,llave)}',0,1,'L')
        aux += 6

    pdf.image(f'hist_{mes}.png',x=43,y=130,w=145)

pdf.output('Mi_Primer_PDF.pdf')

''

In [165]:
import tempfile

temp = tempfile.NamedTemporaryFile()

temp.name

temp.close()

# Reporte en excel

In [255]:
from openpyxl import Workbook
from openpyxl.styles import Font, Color, Alignment, PatternFill, Border
from openpyxl.drawing.image import Image

In [269]:
# Crear un nuevo libro y seleccionar la hoja activa
wb = Workbook()
ws = wb.active

year = df_clean['ATENCION_AÑO'].iloc[0]

# Añadir texto a una celda
ws['A1'] = "Coberturas de atención de las unidades operativas de primer nivel de la dirección distrital 04D02 Montufar Bolivar Salud"

# Establecer fuente, tamaño y color
ws['A1'].font = Font(name='American Typewriter', size = 20, bold=True, italic=True)

meses = df_clean['ATENCION_MES'].unique()

ws['A4'] = f'Consultas de prevención en primer nivel de atención D04D02 Montúfar Bolívar Salud {mes_num(meses[0])}-{mes_num(meses[-1])} {year}'
ws['A4'].font = Font(name='American Typewriter', size = 18, bold=True)

# Añadir imagenes

img = Image('hist.png')
ws.add_image(img,'C6')

etiquetas = ['Menores a 1 año','De 1 a 4 años','De 5 a 9 años','De 10 a 14 años','De 15 a 19 años','De 20 a 64 años','Mayores a 65 años']
llaves = ['Menor_1','1_a_4','5_a_9','10_a_14','15_a_19','20_a_64','Mayor_65']

lista = list(zip(etiquetas,llaves))

aux = 0

ws.column_dimensions['G'].width = 30

for etiqueta,llave in lista:
    ws[f'G{33+aux}'] = f'{etiqueta}'
    ws[f'G{33+aux}'].font = Font(name='American Typewriter', size = 15)
    
    ws[f'H{33+aux}'] = edad_por_rango(0,llave)
    ws[f'H{33+aux}'].font = Font(name='American Typewriter', size = 15)
    aux += 1

aux_meses = 0

for mes in meses:

    ws[f'A{44+aux_meses}'] = f'Consultas de prevención en primer nivel de atención D04D02 Montúfar Bolívar Salud {mes_num(mes)} {year}'
    ws[f'A{44+aux_meses}'].font = Font(name='American Typewriter', size = 18, bold=True)

    # Añadir imagenes

    img = Image(f'hist_{mes}.png')
    ws.add_image(img,f'C{46+aux_meses}')

    aux = 0

    for etiqueta,llave in lista:
        ws[f'G{73+aux+aux_meses}'] = f'{etiqueta}'
        ws[f'G{73+aux+aux_meses}'].font = Font(name='American Typewriter', size = 15)
        
        ws[f'H{73+aux+aux_meses}'] = edad_por_rango(mes,llave)
        ws[f'H{73+aux+aux_meses}'].font = Font(name='American Typewriter', size = 15)
        aux += 1

    aux_meses += 40
'''
# Ajustar el tamaño de la celda
ws.row_dimensions[1].height = 30 # Mi letra es de 20
ws.column_dimensions['E'].width = 60

# Alinear el texto
ws['E1'].alignment = Alignment(horizontal='center',vertical='center')

# Aplicar un color de fondo a una celda
ws['E1'].fill = PatternFill(start_color='050505',end_color='050505',fill_type = 'solid')

img = Image(f'Images/donut_platform_year_{year}.png')
ws.add_image(img,'A4')

img = Image(f'Images/histogram_age_year_{year}.png')
ws.add_image(img,'F4')

plataformas = ['Netflix','Hulu','Prime Video', 'Disney+']
valores = plataforma(year)

edades = edades_permitidas(year)

for i in range(37,41):
    ws[f'C{str(i)}'] = plataformas[i-37]
    ws[f'C{str(i)}'].font = Font(name='American Typewriter', size = 12, bold=True, italic=True,color='005DFF')
    ws[f'D{str(i)}'] = valores[plataformas[i-37]]

for i in range(37,41):
    ws[f'L{str(i)}'] = edades.iloc[i-37]['Age']
    ws[f'L{str(i)}'].font = Font(name='American Typewriter', size = 12, bold=True, italic=True,color='005DFF')
    ws[f'M{str(i)}'] = edades.iloc[i-37]['Amount']
'''
# Guaradar el libro de trabajo
wb.save('reporte_personalizado.xlsx')